In [51]:
from llama_cpp import Llama
llm = Llama(
      model_path="/Users/ashishmadkaikar/code/llama.cpp/models/mistral-7b-v0.1.Q5_K_S.gguf",
      #n_gpu_layers=-1, # Uncomment to use GPU acceleration
      seed=1337, # Uncomment to set a specific seed
      n_ctx=10000, # Uncomment to increase the context window
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/ashishmadkaikar/code/llama.cpp/models/mistral-7b-v0.1.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:

In [ ]:
import json
from chromadb.config import Settings
from chromadb import Chroma
from chromadb.utils import Query, Text

# Load JSON data
with open('/mnt/data/topics/topics.json', 'r') as file:
    data = json.load(file)

# Initialize ChromaDB
chroma = Chroma(Settings(chroma_path='chroma_db'))

# Create a new collection
collection = chroma.create_collection('manuals')

# Helper function to flatten the nested JSON data and insert into ChromaDB
def flatten_and_insert(data, parent_key='', collection=None):
    for k, v in data.items():
        key = f'{parent_key}.{k}' if parent_key else k
        if isinstance(v, dict):
            flatten_and_insert(v, key, collection)
        else:
            doc = {
                'key': key,
                'content': v
            }
            collection.insert(doc)

# Flatten and insert the JSON data
flatten_and_insert(data, collection=collection)

# Define the search query
search_query = "Introduction, Installation, Features, Specifications, Contact"

# Perform the search
query = Query(
    query=search_query,
    k=10,  # limit results to 10
    fields=['key', 'content']
)
results = collection.search(query)

# Print the search results
for i, result in enumerate(results['hits']):
    print(f"Result {i+1}:")
    print(f"Key: {result['key']}")
    print(f"Content: {result['content']}\n")


In [56]:
output = llm(
      "Q: Explain the word espunged A: ", # Prompt
      max_tokens=2000, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion
print(output)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     729.95 ms
llama_print_timings:      sample time =       1.97 ms /    26 runs   (    0.08 ms per token, 13218.10 tokens per second)
llama_print_timings: prompt eval time =     486.66 ms /    10 tokens (   48.67 ms per token,    20.55 tokens per second)
llama_print_timings:        eval time =    1506.21 ms /    25 runs   (   60.25 ms per token,    16.60 tokens per second)
llama_print_timings:       total time =    2001.57 ms /    35 tokens


{'id': 'cmpl-108c2eaf-b067-4a09-b558-1a7c78184fa4', 'object': 'text_completion', 'created': 1719924049, 'model': '/Users/ashishmadkaikar/code/llama.cpp/models/mistral-7b-v0.1.Q5_K_S.gguf', 'choices': [{'text': 'Q: Explain the word espunged A:  It is a fancy way of saying removed.  The definition from my dictionary is: To pull out; to draw forth.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 13, 'completion_tokens': 26, 'total_tokens': 39}}
